# **Agent Training with Supervised Learning**

## Objectives

* Parse hand history data
* Train DQN agent with parsed data
* Save pre-trained model

## Inputs

* outputs/datasets/collection/0.05 - 0.1 - 6max.txt

## Outputs

* Pre-trained model: outputs/models/pretrained

---

# Change working directory

Since jupyter notebooks are in a subfolder we need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'd:\\codeacademy_darbai\\poker_assistant\\poker-assistant-rl\\supervised_learning'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'd:\\codeacademy_darbai\\poker_assistant\\poker-assistant-rl'

# Load Data

In [ ]:
import glob

input_folder = "outputs/datasets/collection"
txt_files = glob.glob(f"{input_folder}/*.txt")
txt_files[0]

---

# Custom Class to Parse Data

In [9]:
import re
from typing import List, Dict

class PokerHandHistoryParser:
    """Parser for poker hand history from a poker website."""
    
    def __init__(self, hero_name: str = 'Hero'):
        """
        Initialize the parser with hand history file.
        
        Args:
            hero_name (str, optional): _description_. Defaults to 'Hero'.
        """
        self.hero_name = hero_name
        self.hands = []  # Will contain parsed hand data
    
    def parse(self):
        """Parse the hand history file into structured data, focusing on hero's gameplay."""
        try:
            with open(txt_files[0], 'r', encoding="utf-8") as file:
                content = file.read()
            
            # Split into individual hands
            hand_texts = self._split_into_hands(content)
            
            # Parse each hand where hero is present
            for hand_text in hand_texts:
                # Only parse hands where the hero is playing
                if self.hero_name in hand_text:
                    parsed_hand = self._parse_hand(hand_text)
                    if parsed_hand:
                        self.hands.append(parsed_hand)
            
            print(f"Successfully parsed {len(self.hands)} hands from history file where {self.hero_name} was playing")
            return self.hands

        except Exception as e:
            print(f"Error parsing hand history: {e}")
            return []
    
    def _split_into_hands(self, content: str) -> List[str]:
        """Split the entire file content into individual hands."""
        hand_pattern = r"Poker Hand #RC\d+: .*?(?=Poker Hand #RC\d+:|$)"
        hands = re.findall(hand_pattern, content, re.DOTALL)
        return hands
    
    def _parse_hand(self, hand_text: str) -> Dict:
        """
        Parse a single hand's text, focusing on hero's perspective.
        
        This needs to be customized for your specific poker site's format.
        """
        hand_data = {
            'hand_id': None,
            'table_name': None,
            'button_seat': None,
            'game_type': None,
            'stakes': None,
            'date_time': None,
            'hero_position': None,
            'hero_seat': None,
            'hero_stack': None,
            'hero_cards': [],  # Hero's hole cards
            'players': {},  # Will contain player names, positions, stacks (but not cards)
            'blinds': {'sb': 0, 'bb': 0},
            'hero_actions': [],  # List of hero's actions in order
            'other_actions': [],  # Actions by other players (for context)
            'community_cards': {
                'flop': [],
                'turn': None,
                'river': None
            },
            'showdown': False,  # Whether hand went to showdown
            'shown_cards': {},  # Cards shown at showdown (if any)
            'hero_result': {
                'won': False,
                'amount': 0,
                'net_win': 0  # Amount won minus amount invested
            }
        }
        
        try:
            # Parse hand ID and basic info - similar to before
            header_match = re.search(r"Poker Hand #(\w+):.*?(\$\d+\.\d+/\$\d+\.\d+).*?- (\d{4}/\d{2}/\d{2} \d{2}:\d{2}:\d{2})", hand_text)
            if header_match:
                hand_data['hand_id'] = header_match.group(1)
                hand_data['stakes'] = header_match.group(2)
                hand_data['date_time'] = header_match.group(3)
            
            # Parse table info
            table_match = re.search(r"Table '([\w\d]+)'.*?(\d+)-max Seat #(\d+) is the button", hand_text)
            if table_match:
                hand_data['table_name'] = table_match.group(1)
                hand_data['max_players'] = int(table_match.group(2))
                hand_data['button_seat'] = int(table_match.group(3))
            
            # Parse player info
            player_pattern = r"Seat (\d+): ([^\(]+) \(\$?([\d.]+) in chips\)"
            for seat, name, stack in re.findall(player_pattern, hand_text):
                name = name.strip()
                player_info = {
                    'seat': int(seat),
                    'stack': float(stack),
                    'position': None  # Will determine position later
                }
                
                # Mark the hero player
                if name == self.hero_name:
                    hand_data['hero_seat'] = int(seat)
                    hand_data['hero_stack'] = float(stack)
                
                hand_data['players'][name] = player_info
            
            # Parse positions based on button
            self._assign_positions(hand_data)
            
            # Parse hero's hole cards
            hero_cards_match = re.search(r"Dealt to " + re.escape('Hero') + r" \[([^\]]+)\]", hand_text)
            if hero_cards_match:
                hand_data['hero_cards'] = hero_cards_match.group(1).split()
                
            # Parse blinds
            sb_match = re.search(r"(\w+): posts small blind \$(\d+\.\d+)", hand_text)
            if sb_match:
                player = sb_match.group(1).strip()
                hand_data['blinds']['sb'] = float(sb_match.group(2))

            bb_match = re.search(r"(\w+): posts big blind \$(\d+\.\d+)", hand_text)
            if bb_match:
                player = bb_match.group(1).strip()
                hand_data['blinds']['bb'] = float(bb_match.group(2))
            
            # Parse actions with particular focus on hero's actions
            self._parse_actions_by_street(hand_text, hand_data)
            
            # Parse community cards
            self._parse_community_cards(hand_text, hand_data)
            
            # Parse showdown results
            self._parse_showdown_and_results(hand_text, hand_data)
            
            return hand_data
            
        except Exception as e:
            print(f"Error parsing individual hand: {e}")
            return None
    
    def _assign_positions(self, hand_data: Dict):
        """Assign poker positions based on button position."""
        # Get the number of players
        num_players = len(hand_data['players'])
        if num_players < 2:
            return
            
        # Get button seat
        btn_seat = hand_data['button_seat']
        
        # Create ordered list of seats
        seats = []
        for name, info in hand_data['players'].items():
            seats.append((info['seat'], name))
        
        # Sort by seat number
        seats.sort()
        
        # Find button index
        btn_idx = -1
        for i, (seat, _) in enumerate(seats):
            if seat == btn_seat:
                btn_idx = i
                break
        
        if btn_idx == -1:
            return
            
        # Assign positions
        positions = ['BTN', 'SB', 'BB', 'UTG', 'MP', 'CO']
        
        for i in range(num_players):
            pos_idx = (btn_idx + i) % num_players
            seat, name = seats[pos_idx]
            
            # Assign position based on table size and index
            if i < len(positions):
                pos = positions[i]
            else:
                pos = f"MP{i-3}"  # MP2, MP3, etc. for larger tables
                
            hand_data['players'][name]['position'] = pos
            
            # Store hero's position
            if name == self.hero_name:
                hand_data['hero_position'] = pos
    
    def _parse_actions_by_street(self, hand_text: str, hand_data: Dict):
        """Parse actions for each betting round, focusing on hero's actions."""
        # Parse preflop actions
        preflop_match = re.search(r"\*\*\* HOLE CARDS \*\*\*(.*?)(?:\*\*\* FLOP \*\*\*|\*\*\* SUMMARY \*\*\*)", hand_text, re.DOTALL)
        if preflop_match:
            self._parse_street_actions(preflop_match.group(1), 'preflop', hand_data)
            
        # Parse flop actions if available
        flop_match = re.search(r"\*\*\* FLOP \*\*\* \[([^\]]+)\](.*?)(?:\*\*\* TURN \*\*\*|\*\*\* SUMMARY \*\*\*)", hand_text, re.DOTALL)
        if flop_match:
            self._parse_street_actions(flop_match.group(2), 'flop', hand_data)
            
        # Parse turn actions if available
        turn_match = re.search(r"\*\*\* TURN \*\*\* \[[^\]]+\] \[([^\]]+)\](.*?)(?:\*\*\* RIVER \*\*\*|\*\*\* SUMMARY \*\*\*)", hand_text, re.DOTALL)
        if turn_match:
            self._parse_street_actions(turn_match.group(2), 'turn', hand_data)
            
        # Parse river actions if available
        river_match = re.search(r"\*\*\* RIVER \*\*\* \[[^\]]+\] \[([^\]]+)\](.*?)(?:\*\*\* SHOWDOWN \*\*\*|\*\*\* SUMMARY \*\*\*)", hand_text, re.DOTALL)
        if river_match:
            self._parse_street_actions(river_match.group(2), 'river', hand_data)
    
    def _parse_street_actions(self, street_text: str, street: str, hand_data: Dict):
        """Parse all actions for a specific betting round."""
        # Extract actions like "PlayerName: raises $X to $Y"
        action_pattern = r"(\w+): (folds|calls|bets|raises|checks)(?: \$?([\d.]+)(?: to \$?([\d.]+))?)?"
        for action_match in re.finditer(action_pattern, street_text):
            player = action_match.group(1).strip()
            action_type = action_match.group(2)
            amount = action_match.group(3)
            raised_to = action_match.group(4)
            
            action_data = {
                'street': street,
                'player': player,
                'action': action_type,
                'amount': float(amount) if amount else None,
                'raised_to': float(raised_to) if raised_to else None
            }
            
            # Separate hero actions from other actions
            if player == self.hero_name:
                hand_data['hero_actions'].append(action_data)
            else:
                hand_data['other_actions'].append(action_data)
    
    def _parse_community_cards(self, hand_text: str, hand_data: Dict):
        """Parse community cards for flop, turn, and river."""
        # Parse flop
        flop_match = re.search(r"\*\*\* FLOP \*\*\* \[([^\]]+)\]", hand_text)
        if flop_match:
            hand_data['community_cards']['flop'] = flop_match.group(1).strip().split()
            
        # Parse turn
        turn_match = re.search(r"\*\*\* TURN \*\*\* \[[^\]]+\] \[([^\]]+)\]", hand_text)
        if turn_match:
            hand_data['community_cards']['turn'] = turn_match.group(1).strip()
            
        # Parse river
        river_match = re.search(r"\*\*\* RIVER \*\*\* \[[^\]]+\] \[([^\]]+)\]", hand_text)
        if river_match:
            hand_data['community_cards']['river'] = river_match.group(1).strip()
    
    def _parse_shown_cards(self, hand_text: str, hand_data: Dict):
        """Parse hands shown at the end of hand."""
        if "*** SHOWDOWN ***" in hand_text:
            hand_data['showdown'] = True
            
            # Parse shown cards
            shown_cards_pattern = r"(\w+): shows \[([^\]]+)\]"
            for player, cards in re.findall(shown_cards_pattern, hand_text):
                hand_data['shown_cards'][player.strip()] = cards.strip().split()
        
    def _parse_showdown_and_results(self, hand_text: str, hand_data: Dict):
        # Find Hero's blinds
        small_blind_match = re.search(r"Hero: posts small blind \$(\d+\.\d+)", hand_text)
        big_blind_match = re.search(r"Hero: posts big blind \$(\d+\.\d+)", hand_text)
        
        small_blind = float(small_blind_match.group(1)) if small_blind_match else 0.0
        big_blind = float(big_blind_match.group(1)) if big_blind_match else 0.0

        # Find Hero's raises (only the final value)
        hero_raises = re.findall(r"Hero: raises \$(?:[\d.]+) to \$(\d+\.\d+)", hand_text)
        total_raise = float(hero_raises[-1]) if hero_raises else 0.0

        # Find Hero's calls and bets
        hero_bets_calls = re.findall(r"Hero: (bets|calls) \$(\d+\.\d+)", hand_text)
        total_bet_call = sum(float(amount) for _, amount in hero_bets_calls)

        # Determine Hero's total bet amount
        if hero_raises:
            total_bet = total_raise  # Use final raise value
            if hero_bets_calls:  # Add any bets/calls AFTER a raise
                total_bet += total_bet_call
        else:
            total_bet = total_bet_call  # If no raise, just sum bets/calls

        # If Hero did not bet/raise, add blinds
        if not (hero_bets_calls or hero_raises):
            total_bet += small_blind + big_blind

        # Find Hero's winnings
        uncalled_bet_match = re.search(r"Uncalled bet \(\$(\d+\.\d+)\) returned to Hero", hand_text)
        collected_pot_match = re.search(r"Hero collected \$(\d+\.\d+) from pot", hand_text)

        uncalled_bet = float(uncalled_bet_match.group(1)) if uncalled_bet_match else 0.0
        collected_pot = float(collected_pot_match.group(1)) if collected_pot_match else 0.0

        total_won = round(uncalled_bet + collected_pot, 2)
        net_result = round(total_won - total_bet, 2)
        
        hand_data['hero_result']['won'] = True if total_won > 0 else False
        hand_data['hero_result']['amount'] = total_won
        hand_data['hero_result']['net_win'] = net_result

In [ ]:
import json
parser = PokerHandHistoryParser()
parser.parse()
print(json.dumps(parser.hands[11], indent=4))